### GWU STAT 4197/6197
##### Week 6 SAS Code Examples: PROC SQL Basics
(Source: SAS Documentation for Code Explanation)

### SQL procedure 
* begins with PROC SQL statement
* ends with a QUIT statement

### SELECT statement and its clauses

* The SELECT statement retrieves and displays data.
* A SELECT statement can be used to query up to 256 tables.
* The results of the SELECT statement are written to the default output destination.
* The SELECT clause specifies the columns and column order.
* An asterisk in the SELECT caluse means all columns.
* The FROM clause specifies the data sources.

The following SQL code is eqivalent to the PROC PRINT step with a VAR statement in it.


In [1]:
proc sql;
select * 
 From sashelp.class;
quit;

SAS Connection established. Subprocess id is 6668



Name,Sex,Age,Height,Weight
Alfred,M,14,69,112.5
Alice,F,13,56.5,84
Barbara,F,13,65.3,98
Carol,F,14,62.8,102.5
Henry,M,14,63.5,102.5
James,M,12,57.3,83
Jane,F,12,59.8,84.5
Janet,F,15,62.5,112.5
Jeffrey,M,13,62.5,84
John,M,12,59,99.5


In [1]:
proc sql inobs=4;
select name, weight format = 6.1
From sashelp.class;
quit;

SAS Connection established. Subprocess id is 9680



Name,Weight
Alfred,112.5
Alice,84.0
Barbara,98.0
Carol,102.5


### Calculating Columns and Column Alias in PROC SQL

In [3]:
proc sql;
select name, weight format = 6.1,
round((weight*703.069)/(height**2), .01) as bmi
From sashelp.class;
quit;

Name,Weight,bmi
Alfred,112.5,16.61
Alice,84.0,18.5
Barbara,98.0,16.16
Carol,102.5,18.27
Henry,102.5,17.87
James,83.0,17.77
Jane,84.5,16.61
Janet,112.5,20.25
Jeffrey,84.0,15.12
John,99.5,20.1


### Subsetting with the WHERE Clause
* The WHERE clause can be used to specifiy a condition that the data must satisfy before being selected.

In [7]:
proc sql;
create table class as
select name, sex, weight format=6.1, height format=6.1,
round((weight*703.069)/(height**2), .1) as bmi
From sashelp.class;
select * 
 from class
 where sex = 'M';
quit;

Name,Sex,Weight,Height,bmi
Alfred,M,112.5,69.0,16.6
Henry,M,102.5,63.5,17.9
James,M,83.0,57.3,17.8
Jeffrey,M,84.0,62.5,15.1
John,M,99.5,59.0,20.1
Philip,M,150.0,72.0,20.3
Robert,M,128.0,64.8,21.4
Ronald,M,133.0,67.0,20.8
Thomas,M,85.0,57.5,18.1
William,M,112.0,66.5,17.8


### Subsetting with Calculated Values in the WHERE Clause

In [10]:
proc sql;
select name, sex, weight format=6.1, height format=6.1,
round((weight*703.069)/(height**2), .1) as bmi
From sashelp.class
 where calculated bmi ge 20;
quit;

Name,Sex,Weight,Height,bmi
Janet,F,112.5,62.5,20.2
John,M,99.5,59.0,20.1
Philip,M,150.0,72.0,20.3
Robert,M,128.0,64.8,21.4
Ronald,M,133.0,67.0,20.8


### GROUP BY Clause
* classifies  the data into groups based on the values of one or more columns
* calculates statistics for each unique value of the grouping columns

In the following code, the GROUP BY clause classifies the data based on the values 
of the MAKE column and calculate the mean INVOICE price for each 
unique value of the grouping column (i.e., MAKE). 

The ORDER BY clause order the results in descending order.

In [1]:
*Ex6_sql_moreCode.sas (Part 13);
proc sql;
 select make, 
  mean(invoice) as mean_invoice_price format=dollar8.
   From sashelp.cars
   group by make
   order by mean_invoice_price desc;
quit;

SAS Connection established. Subprocess id is 9660



Make,mean_invoice_price
Porsche,"$73,663"
Mercedes-Benz,"$56,453"
Jaguar,"$56,098"
Cadillac,"$46,427"
Hummer,"$45,815"
Land Rover,"$41,851"
BMW,"$39,621"
Audi,"$39,330"
Lincoln,"$39,136"
Lexus,"$38,760"


### HAVING Caluse and CALCULATED Keyword

In the following code, the HAVING clause subsets groups based on the expression value.
Here we use a column alias to refer to a calculated value. and hence, use 
the CALCULATED keyword with the alias to inform PROC SQL that 
the value is calculated within the query (SAS Documentation).


In [29]:
*Ex6_sql_moreCode.sas (Part 14); 
proc sql ; 
select name
       ,pop
       ,pop/sum(pop) as percent_pop 
          format=percent7.2
  from  sashelp.demographics
   having calculated percent_pop >0.02
   order by percent_pop desc;
quit;

GLC Country Name,Population (2005),percent_pop
CHINA,"1,323,344,591",19.8%
INDIA,"1,103,370,802",16.5%
UNITED STATES,"298,212,895",4.47%
INDONESIA,"222,781,487",3.34%
BRAZIL,"186,404,913",2.79%
PAKISTAN,"157,935,075",2.37%
RUSSIA,"143,201,572",2.15%
BANGLADESH,"141,822,276",2.13%


### HAVING Clause
* subsets groups based on the expression value.
* is  processed after the GROUP BY clause and determines  which groups are displayed.

In [11]:
proc sql;
select make 
       ,count(make) as N
       ,Avg(invoice) as avg_invoice format=dollar7.
From sashelp.cars
 group by make
 having Avg(invoice) ge 50000
 order by avg_invoice desc;
quit;

Make,N,avg_invoice
Porsche,7,"$73,663"
Mercedes-Benz,26,"$56,453"
Jaguar,12,"$56,098"


In [1]:
*Ex9_Dictionary_column.sas (Part 1);
OPTIONS nocenter nodate nonumber ;
Title1 'Get column names and column type from a particular SAS Table';
proc sql;
 SELECT NAME, TYPE, LABEL  
  FROM DICTIONARY.COLUMNS  
   WHERE UPCASE(LIBNAME)="SASHELP" AND  
         UPCASE(MEMNAME)="HEART";  
QUIT;


SAS Connection established. Subprocess id is 9068



Column Name,Column Type,Column Label
Status,char,
DeathCause,char,Cause of Death
AgeCHDdiag,num,Age CHD Diagnosed
Sex,char,
AgeAtStart,num,Age at Start
Height,num,
Weight,num,
Diastolic,num,
Systolic,num,
MRW,num,Metropolitan Relative Weight


### Use the INSERT statement with the VALUE clause to add rows to a data table


In [27]:
*** Acknowlegements: Some of the code idea  (INSERT, VALUE, SET,  
      and UPDATE clauses) is obtained from Martha Messineo (2017);

*** Use the INSERT clause to create a data table;
options nocenter nonumber nodate nonotes nosource;
proc sql;
Create table have(ID varchar(6), stype varchar(13), Score float);
Insert into have(id, stype, score)
    Values('S00003', 'Graduate', 19)
    Values('S00002', 'Graduate', 14)
    Values('S00007', 'Graduate', 18)
    Values('S00008', 'Graduate', 16)
    Values('M00004', 'Undergraduate', 13)
    Values('E00005', 'Undergraduate', 18)
    Values('M00022', 'Undergraduate', 17)
    Values('E00035', 'Undergraduate', 16)
    ;
 title;   
 select * from have; 
quit;

ID,stype,Score
S00003,Graduate,19
S00002,Graduate,14
S00007,Graduate,18
S00008,Graduate,16
M00004,Undergraduate,13
E00005,Undergraduate,18
M00022,Undergraduate,17
E00035,Undergraduate,16


In [28]:
*Ex6_sql_moreCode.sas (Part 3);
proc sql;
create table class AS
select * 
  from sashelp.class;
Insert into class
  values('Amy', 'F', 14,64, 107);
 select *
    from class;
quit;

Name,Sex,Age,Height,Weight
Alfred,M,14,69,112.5
Alice,F,13,56.5,84
Barbara,F,13,65.3,98
Carol,F,14,62.8,102.5
Henry,M,14,63.5,102.5
James,M,12,57.3,83
Jane,F,12,59.8,84.5
Janet,F,15,62.5,112.5
Jeffrey,M,13,62.5,84
John,M,12,59,99.5


In [29]:
*Ex6_sql_moreCode.sas (Part 6);
*** Use the UPDATE clause to conditionally set a value of a particular
variable to another value;
proc sql;
  create table new_class AS
   select *
   from sashelp.class;
   update new_class
    set age=age+1
    where sex='F';
    select *
    from new_class;
quit;


Name,Sex,Age,Height,Weight
Alfred,M,14,69,112.5
Alice,F,14,56.5,84
Barbara,F,14,65.3,98
Carol,F,15,62.8,102.5
Henry,M,14,63.5,102.5
James,M,12,57.3,83
Jane,F,13,59.8,84.5
Janet,F,16,62.5,112.5
Jeffrey,M,13,62.5,84
John,M,12,59,99.5


The following two code blocks (PROC SORT/PROC PRINT and PROC SQL) 
    provide the same results).

In [26]:
*Ex6_sql_moreCode.sas (Part 11);
proc sort data=sashelp.cars 
       (where=(make='BMW')) nodupkey
  out=distinct_make_type (keep=make type);
  by make type;
run;
proc print data=distinct_make_type noobs;
run;

Make,Type
BMW,SUV
BMW,Sedan
BMW,Sports
BMW,Wagon


In [24]:

*Ex6_sql_moreCode.sas (Part 9);
data _null_;
  set sashelp.cars end=eof;
  count+1;
  if eof then putlog count=;
run;


In [25]:
*Ex6_sql_moreCode.sas (Part 10);
proc sql;
 select count(*) as total_rows
 From sashelp.cars;
quit;

total_rows
428


In [1]:
*Ex1_concat_interleave.sas ;
ods html close;
options nocenter nonumber nodate nosource nonotes;
DATA D1;
  INPUT CustID Month $ purchased_amount;
  DATALINES; 
  11 Jan 237.4 
  12 Jan 249.2 
  13 Jan 227.7 
  ;


SAS Connection established. Subprocess id is 11356



In [2]:
ods html close;
options nocenter nonumber nodate nosource nonotes;
DATA D2;
 INPUT CustID Month $ purchased_amount;
 DATALINES;
 11 Feb 288.2 
 12 Feb 221.7 
 13 Feb 274.4  
 14 Feb 222.9
;

In [8]:
options nocenter nonumber nodate;

proc sql;
 create table concat_sql as
 select * from D1
   union 
 select * from D2
 order by Month desc;
 title1 'Vertical Joining Using PROC SQL';
 select * from concat_sql;
quit;

CustID,Month,purchased_amount
11,Jan,237.4
12,Jan,249.2
13,Jan,227.7
11,Feb,288.2
12,Feb,221.7


### UNION operator with PROC SQL

The default behavior of the UNION operator is that the duplicate rows are removed from the final results. 
Due to ALL modifier,duplicate results are not removed. 

With the UNION operator, column names in the final result set are determined by the first result set.

However, due to CORRESPONDING modifier, columns are matched by name and nonmatching columns are removed 
from the intermediate result set (SAS Documentation).

In [13]:
options nocenter nonumber nodate;
proc sql;
 create table concat_sql_i as
 select * from D1
   union 
 select * from D2
 order by CustID, Month desc;
  title1
 'Vertical Joining Using PROC SQL /Interleaved';
 select * from concat_sql_i;
quit;

CustID,Month,purchased_amount
11,Jan,237.4
11,Feb,288.2
12,Jan,249.2
12,Feb,221.7
13,Jan,227.7
13,Feb,274.4
14,Feb,222.9


In [2]:
options nocenter nodate nonumber;
DATA BIRTH;
  INPUT id $ dob : mmddyy.;
  FORMAT dob  mmddyy10.;
  DATALINES; 
03 03/31/1944 
04 08/11/1950
01 01/09/1954 
02 09/12/1959 
05 07/18/1941
;
PROC SORT data=BIRTH; by id; 
title1 'BIRTH File - Listing'; 
PROC PRINT data=BIRTH noobs;  run;


id,dob
01,01/09/1954
02,09/12/1959
03,03/31/1944
04,08/11/1950
05,07/18/1941


In [3]:
DATA DEATH;
input id $ dod : mmddyy.;
FORMAT dod mmddyy10.;
DATALINES;
07 12/31/2011 
08 02/14/2012
04 12/31/2010 
05 12/12/2012 
06 12/29/2011 
; 
PROC SORT data=DEATH; by id; 
title1 'DEATH File - Listing';
PROC PRINT data=DEATH noobs;  run;

id,dod
04,12/31/2010
05,12/12/2012
06,12/29/2011
07,12/31/2011
08,02/14/2012


### Full Join in PROC SQL
* Includes all rows from both tables even if there are no matching rows in either table
* does not overlay same-name columns.

In [19]:
options nocenter nodate nonumber;
proc sql;
title 'Full Join/PROC SQL does not overlay same-name columns';
select * 
    from BIRTH b full join DEATH d
      on b.id = d.id;
quit;


id,dob,id,dod
01,01/09/1954,,.
02,09/12/1959,,.
03,03/31/1944,,.
04,08/11/1950,04,12/31/2010
05,07/18/1941,05,12/12/2012
,.,06,12/29/2011
,.,07,12/31/2011
,.,08,02/14/2012


### COALESCE Function
* does overlays columns
* returns the value of the first nonmissing argument

In [22]:
options nocenter nodate nonumber;
proc sql;
title1 'Full Join/PROC SQL does overlay same-name columns with COALESCE function';
select coalesce(b.ID, d.ID) as ID, dob, dod
    from BIRTH b full join DEATH d
      on b.id = d.id;
quit;

ID,dob,dod
01,01/09/1954,.
02,09/12/1959,.
03,03/31/1944,.
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012
06,.,12/29/2011
07,.,12/31/2011
08,.,02/14/2012


### Inner Join
* returns only matching rows

### COALESCE Function
* does overlays the same-name columns
* returns the value of the first nonmissing argument

In [20]:
options nocenter nodate nonumber;
proc sql;
title1 'Inner Join/PROC SQL';
select coalesce(b.id, d.id) as id,
       b.dob, d.dod 
   from BIRTH as b
   inner join  DEATH as d
     on b.id = d.id;
quit;


id,dob,dod
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012


### Inner Join 
* specifies the matching criteria in the WHERE clause (as an alternative)

In [21]:
options nocenter nodate nonumber;
proc sql;
title1 'Inner Join 2 /PROC SQL';
select b.id, b.dob, d.dod 
   from BIRTH as b,
        DEATH as d
     where b.id = d.id;
quit;

id,dob,dod
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012


### Exact match with subquery in PROC SQL

In [7]:
options nocenter nodate nonumber;
proc sql;
  select id, dob
  from birth
  where id in (select id from death);
quit;


id,dob
04,08/11/1950
05,07/18/1941


### Left Join
* returns all matching and non-matching rows from the left table
* returns matching rows from the right table 

In [24]:
options nocenter nodate nonumber;
proc sql;
title1 'Left Join/PROC SQL';
select coalesce(b.id, d.id) as id,
       b.dob, d.dod 
   from BIRTH as b
   left join  DEATH as d
     on b.id = d.id;
quit;

id,dob,dod
01,01/09/1954,.
02,09/12/1959,.
03,03/31/1944,.
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012


### Right Join
* returns all matching and non-matching rows from the right table
* returns matching rows from the left table 

In [23]:
options nocenter nodate nonumber;
proc sql;
title1 'Right Join/PROC SQL';
select coalesce(b.id, d.id) as id,
       b.dob, d.dod 
       from BIRTH as b right join DEATH as d
      on b.id = d.id;
quit;

id,dob,dod
04,08/11/1950,12/31/2010
05,07/18/1941,12/12/2012
06,.,12/29/2011
07,.,12/31/2011
08,.,02/14/2012


### SQL subquery

In [28]:
options nocenter nodate nonumber;
*PROC SQL subquery finding BIRTH IDs that are not in the DEATH file; 
proc sql;
title1 'SQL subquery - Finding BIRTH IDs that are not in the DEATH file';
  select id, dob
  from birth
  where id not in(select id from death);
quit;


id,dob
01,01/09/1954
02,09/12/1959
03,03/31/1944


In [31]:
options nocenter nodate nonumber;
*PROC SQL subquery finding DEATH IDs that are not in the BIRTH file; ; 
proc sql;
title1 'SQL subquery - Finding DEATH IDs that are not in the BIRTH file';
  select id, dod 
  from death
  where id not in(select id from birth);
quit;

id,dod
06,12/29/2011
07,12/31/2011
08,02/14/2012


### Creating Macro Variables Using PROC SQL

* Get the variable names into a macro variable using PROC SQL
* Extract the values of macro variable


In [33]:
*Ex9_Dictionary_column.sas (Part 3);
OPTIONS nocenter nodate nonumber nosource ;
ods html close;
proc sql noprint; 
     select name into : COLNAMES separated by ' ' 
     from dictionary.columns 
     where LIBNAME='SASHELP' and memname = UPCASE('class');
 quit; 

 %put  Get a horizontal list of column names of a SAS table: &COLNAMES;


In [37]:
*Ex9_Dictionary_column.sas (Part 2);
Title 'Get the column names that are the same in all SAS tables in the same library';
OPTIONS nocenter nodate nonumber ;
proc sql;
 SELECT MEMNAME 'Table Names', NAME
  FROM DICTIONARY.COLUMNS  
   WHERE UPCASE(LIBNAME)="SASHELP" AND  
         UPCASE(NAME)="WEIGHT";  
QUIT;

Table Names,Column Name
BWEIGHT,Weight
CARS,Weight
CLASS,Weight
CLASSFIT,Weight
FISH,Weight
GRIDDED,Weight
HEART,Weight


### SASHELP.VTABLE 

* has the information on the number of   numeric variables and the number of character variables
  for every SAS dataset: num_character, num_numeric.



In [34]:
 *Ex9_Dictionary_column.sas (Part 4);
OPTIONS nocenter nodate nonumber nonotes nosource ;
ods html close;
%let lref=SASHELP;
%let dsn=CLASS;

proc sql noprint;
select num_character, num_numeric into :n_char_x, :n_num_x
from sashelp.vtable
where libname = upcase("&lref.") and upcase(memname) = upcase("&dsn.");
quit;

%put Number of character variables = &n_char_x;
%put Number of numeric variables = &n_num_x;


In [36]:
 *Ex9_Dictionary_column.sas (Part 6);
 OPTIONS nocenter nodate nonumber nonotes nosource ;
ods html close;

* Data Step Approach;
Data test;
  set sashelp.cars;
  array nums(*) _numeric_;
  array chrs(*) _character_;
     call symputx('nb1',dim(nums),'G');
     call symputx('nb2',dim(chrs),'G');
run;

%put Number of numeric variables = &nb1;
%put Number of numeric variables = &nb2;


In the sample program below, we want to create a macro variable whose values have the suffix “_Status” from the dataset “HEART” saved in the “SASHELP” library. 

[A Simple Macro to Select Various Variables Lists ](https://www.pharmasug.org/proceedings/2015/QT/PharmaSUG-2015-QT10.pdf)

In [38]:
*Ex9_Dictionary_column.sas (Part 7);
OPTIONS nocenter nodate nonumber nonotes nosource ;
ods html close;
proc sql noprint; 
 select name into :varlist_status separated by ' '
 from dictionary.columns
 where libname='SASHELP' and memname='HEART'
 and name like "%#_Status" escape '#';
 quit;
%put &varlist_status;

In [46]:
*Ex9_Dictionary_column.sas (Part 7);
OPTIONS nocenter nodate nonumber nonotes nosource ;
ods html close;
proc sql noprint; 
 select name into :varlist separated by ' '
 from dictionary.columns
 where libname='SASHELP' and memname='HEART'
 and name like '%_Status%';
 quit;
%put &varlist;